In [1]:
# pip install tflearn

In [2]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Used for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\varun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [6]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
277 documents
42 classes ['about', 'art', 'books', 'bye', 'career', 'cooking', 'education', 'emotion', 'environment', 'fashion', 'finance', 'fitness', 'food', 'gardening', 'goodbye', 'greetings', 'health', 'help', 'hi', 'history', 'joke', 'languages', 'mindfulness', 'movies', 'music', 'name', 'news', 'opentoday', 'parenting', 'pets', 'photography', 'psychology', 'relationships', 'science', 'self-improvement', 'sports', 'technology', 'thanks', 'time', 'travel', 'weather', 'writing']
246 unique stemmed words ['!', "'m", "'s", '.', 'a', 'about', 'act', 'adv', 'afternoon', 'an', 'angry', 'anxy', 'any', 'ap', 'apart', 'apprecy', 'ar', 'art', 'assist', 'athlet', 'auth', 'be', 'beauty', 'becom', 'begin', 'behavy', 'benef', 'benefit', 'bet', 'book', 'bree', 'budget-friendly', 'bye', 'can', 'car', 'cold', 'common', 'commun', 'conceiv', 'conserv', 'contribut', 'cook', 'cuisin', 'cur', 'dai', 'day', 'delicy', 'design', 'destin', 'develop', 'direct

In [7]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [8]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")
# tf.reset_default_graph()
tf.compat.v1.reset_default_graph()

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [9]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training....


In [10]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [11]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')


Training Step: 34999  | total loss: 0.07780 | time: 0.089s
| Adam | epoch: 1000 | loss: 0.07780 - acc: 0.9457 -- iter: 272/277
Training Step: 35000  | total loss: 0.07003 | time: 0.091s
| Adam | epoch: 1000 | loss: 0.07003 - acc: 0.9511 -- iter: 277/277
--
Saving the Model.......
INFO:tensorflow:C:\Users\varun\0 Python Notebooks and Data\NLP Assignment\ChatBot_Tensorflow_NLP-master\ChatBot_Tensorflow_NLP-master\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [13]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from C:\Users\varun\0 Python Notebooks and Data\NLP Assignment\ChatBot_Tensorflow_NLP-master\ChatBot_Tensorflow_NLP-master\model.tflearn


In [14]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [15]:
# das

In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- hey
Greetings!
You- i'm not feeling well
If you have a medical question or are seeking remedies for a common cold, it's best to consult with a healthcare professional. They can provide accurate medical advice and suggest appropriate remedies based on your symptoms.
You- do you have any medical advice
Sure! Are you looking for resources in a specific subject or area of study?
You- health tips
To maintain good health, it's important to follow a balanced diet, engage in regular physical activity, get enough restful sleep, and manage stress. However, for personalized advice, I recommend consulting with a healthcare professional.
You- thanks
You're welcome! If there's anything else I can do for you, let me know.
You- do you know any jokes?
Did you hear about the mathematician who's afraid of negative numbers? He'll stop at nothing to avoid them!
You- tell me about a good movie
Sure! How about 'Inception', 'The Dark Knight', or 'Interstellar'?
You- can you give me some career tips
I can